In [1]:
# Nötige Imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer
import tensorflow as tf
tf.keras.__version__
import torch
from transformers import pipeline
import glob
from tqdm import tqdm
import pandas as pd
import fileinput

# GPU benutzen
try:
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print("Cuda available: ", torch.cuda.is_available())
    print("Cuda current device: ", torch.cuda.current_device())
    print("Cuda device s: ", torch.cuda.device(0))
    print("Cuda device count: ", torch.cuda.device_count())
    print("Cuda device name: ", torch.cuda.get_device_name(0))
    gpus = tf.config.experimental.list_physical_devices('GPU')
    memli=4000
    if gpus:
        try:
            tf.config.experimental.set_virtual_device_configuration(
            gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memli)])
            print("Memory limit for gpu is", memli,"mb")
        except RuntimeError as e:
            print(e)
except RuntimeError as e:
    print("no gpu")

Num GPUs Available:  0
Cuda available:  True
Cuda current device:  0
Cuda device s:  <torch.cuda.device object at 0x7fdc2f29db50>
Cuda device count:  1
Cuda device name:  NVIDIA GeForce RTX 3060


2022-06-23 19:56:47.175192: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 19:56:47.208103: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-06-23 19:56:47.208119: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
tqdm.pandas()

In [3]:
# Tokenizer importieren, Modell erstellen
tokenizer2 =  RobertaTokenizer.from_pretrained("bhadresh-savani/roberta-base-emotion", do_lower_case=True)

# BERT-Pipeline erstellen
classifier = pipeline("text-classification",model='bhadresh-savani/roberta-base-emotion', return_all_scores=True)


In [4]:
# Funktion zur Ermittlung des Durchschnitts der Scores
def calc_scores (temp_list_pred):
    score_sadness = 0
    score_joy = 0
    score_love = 0
    score_anger = 0
    score_fear = 0
    score_suprise = 0

    for x in temp_list_pred:
        score_sadness += x[0][0]["score"]
        score_joy += x[0][1]["score"]
        score_love += x[0][2]["score"]
        score_anger += x[0][3]["score"]
        score_fear += x[0][4]["score"]
        score_suprise += x[0][5]["score"]

    score_sadness = score_sadness/le
    score_joy = score_joy/le
    score_love = score_love/le
    score_anger = score_anger/le 
    score_fear = score_fear/le
    score_suprise = score_suprise/le
    list_all = [score_sadness,score_joy,score_love,score_anger,score_fear,score_suprise]
    
    return(list_all)

### Modell anwenden

In [6]:
# Variablen erstellen
temp_list_pred=[]
list_pred = []

# Loop durch in Daten-Pipeline erstellte Exports
for files in glob.glob("/home/keks/Documents/GitHub/nlp_wapo/data/v6/rdy/16/test/*.json"):
    print("reading from: "+ files)
    # DF einlesen
    df = pd.read_json(files)
    
    # DF kürzen für Prototyp
    df = df[:5]
    
    # Loop durch DF
    for lines in df.content:
        
        # Wenn über 500 Chars, dann durchreichen in einzelnen Teilen zur Analyse-Pipeline
        if len(lines) >= 500:

            # Temporäre Variablen
            le = int(len(lines)/500)
            start = 0
            end = 500
            i = 1 

            while i <= le:
                prediction = classifier(lines[start:end],)
                temp_list_pred.append(prediction)
                start = end
                end = end + 500
                i+=1

            prediction = classifier(lines[start:len(lines)],)
            temp_list_pred.append(prediction)

        else:
            prediction = classifier(lines,)
            temp_list_pred.append(prediction)

        # Ermittlung der Score Durchschnitte
        res = calc_scores(temp_list_pred)
        list_pred.append(res)
        
    # Predictions an DF anhängen
    df["scores"] = list_pred
    
print("rdy")    

reading from: /home/keks/Documents/GitHub/nlp_wapo/data/v6/rdy/16/test/4_rdy.json
rdy


In [7]:
df

,Kategorie,Titel,Autor,Datum,content,scores
43068,Capital Business,Deltek: High-value competitions coming in the ...,Ashley Bergander,2014-09-07T19:33:27Z,Some major federal contracts are up for compet...,"[0.030104692559689283, 0.9396944791078568, 0.0..."
43069,Capital Business,District company spins silver into yarn,Abha Bhattarai,2014-09-07T19:38:51Z,Laurie Gonyea has created a 12-step program to...,"[0.04154728923458606, 1.2973899831995368, 0.02..."
43070,Capital Business,Washington-area business diary for week of Sep...,Shawn Selby,2014-09-07T19:39:47Z,"""Short takes on the weeks announcements and de...","[0.13987362353752056, 4.427117166419824, 0.082..."
43071,Capital Business,Commentary: Our brave new world of communication,Greg Kihlström,2014-09-07T19:42:04Z,"More than any other skill in business, none ma...","[0.0644283319124952, 2.7113710632547736, 0.041..."
43072,Capital Business,Career Coach: Tips for getting caught up,Joyce E.A. Russell,2014-09-07T19:44:04Z,"We just celebrated Labor Day, a tribute to the...","[0.224168244516477, 6.395383497700095, 0.09251..."


In [9]:
df.scores[43068]

[0.030104692559689283,
 0.9396944791078568,
 0.015655590686947107,
 0.1993081532418728,
 0.06142737390473485,
 0.003809720539720729]

0	sadness <br>
1 	joy <br>
2	love <br>
3	anger <br>
4	fear <br>
5	suprise <br>


In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [5]:
for files in glob.glob("/home/keks/Documents/GitHub/nlp_wapo/data/v6/rdy/16/test/*.json"):
    print("reading from: "+ files)

reading from: /home/keks/Documents/GitHub/nlp_wapo/data/v6/rdy/16/test/4_rdy.json
